### Data Loading


In [1]:
import pandas as pd
from oracle import oracleTopd
query = ''
data = oracleTopd('')

### Doc2Vec으로 임베딩하기

- 전처리 없이 샘플 10000개 임베딩
- 1. recipe_step을 형태소 분석기로 전처리
- 2. gensim doc2vec으로 유사도 분석

In [2]:
data.columns

Index(['recipe_url', 'recipe_title', 'recipe_summary1', 'recipe_summary2',
       'recipe_summary3', 'recipe_ingredients', 'recipe_step', 'recipe_tag',
       'recipe_comment', 'recipe_view', 'recipe_category_type',
       'recipe_category_method', 'recipe_category_situation'],
      dtype='object')

In [3]:
import random 
import numpy as np

np.random.seed(77)
toy = data[["recipe_title", "recipe_step", "recipe_ingredients"]].sample(1000) # toy 데이터 만들기 표본 1000개

In [4]:
data["recipe_ingredients"]

0         {'재료': ['애호박1개', '다진마늘0.5큰술', '소금0.5큰술', '참기름1...
1         {'재료': ['건새우50g', '물1/4컵', '마늘종300g', '소금1/3큰술...
2         {'재료': ['부추1줌', '달걀4개', '식용유2큰술', '밥2공기'], '양념...
3         {'재료': ['베이컨5~6줄', '양파1/2개', '미니새송이버섯7개', '다진마...
4         {'재료': ['깻잎', '대파', '다진마늘1T', '국간장1T', '참기름1T'...
                                ...                        
168835                                                   {}
168836                                                   {}
168837                                                   {}
168838                                                   {}
168839                                                   {}
Name: recipe_ingredients, Length: 168840, dtype: object

In [5]:
toy #레시피 타이틀, 레시피 스텝으로 구성

,recipe_title,recipe_step,recipe_ingredients
25036,떡잡채- 수미네반찬 10회,"['#1 소고기(150g)은 양조간장(1큰술), 설탕(1작은술), 마늘(1큰술), ...","{'떡잡채재료': ['소고기150g', '양조간장1큰술', '설탕1작은술', '마늘..."
26159,오징어볶음과 오징어덮밥으로도 간단하게~,['#1 오징어를 지난번 다리를 다 먹어버렸어요.그래서 오늘은 이렇게 몸통만 2마리...,"{'재료': ['오징어몸통2마리', '양파1개', '당근1/4개', '청량고추1개'..."
20386,순정 느타리버섯으로 만드는 버섯 겉절이,"['#1 순정 느타리버섯은 적당한 크기로 손으로 찢어 준다.', '#2 찢은 버섯은...","{'재료': ['느타리버섯100g', '소금1/2Ts', '양파1/4개', '고춧가..."
113414,[크림 파스타] 정통 카르보나라(까르보나라),"['#1 그릇에 계란과 계란 노른자, 치킨 스톡, 페코리노 로마노, 후추를 넣고 섞...","{'재료': ['스파게티120g', '베이컨3줄', '계란1개', '계란노른자2개'..."
21341,새우계란찜,['#1 새우는 머리를 떼고 몸통 부분의 껍질을 벗겨줍니다. 그리고 새우 등쪽과 배...,"{'재료': ['순두부', '새우3~4마리', '계란2개', '양파', '당근', ..."
...,...,...,...
94272,굴소스 어묵볶음,"['#1 파프리카는 먹기 좋은 크기로 썰어주세요.', '#2 달군 팬에 식용유 두르...","{'재료': ['요리어묵200g', '파프리카약간', '양파약간'], '양념': [..."
168638,겨울을 더 건강하게! 겨울제철 음식 최고의 궁합♡,[],{}
143248,달콤한 애플 군만두,"['#1 사과는 깨끗이 씻어 껍질채 잘게 썰어 준비해주세요~', '#2 작은 냄비나...","{'재료': ['사과1개', '버터15g', '흑설탕1숟갈', '계피가루1/4작은술..."
43233,시래기된장국,"['#1 엄마가 삶아주신 무청시래기입니다', '#2 집에가져와서 작에 썰어줘요반은 ...","{'재료': ['무청시래기', '대파', '다진마늘'], '양념': ['된장1T',..."


In [6]:
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

okt = Okt()
# kkma = Kkma()
# komoran = Komoran()
# hannanum = Hannanum()

In [7]:
import re
tagged_corpus_list = []
cleaned_tagged_corpus_list = []

for index, row in tqdm(toy.iterrows(), total=len(toy)):
  cleaned_text = re.sub(r'[^A-Za-z0-9가-힣\s]+', '', row['recipe_step'])
  words = okt.morphs(cleaned_text)
  tag = row['recipe_title']
  tagged_corpus_list.append(TaggedDocument(words=words, tags=[tag]))


100%|██████████| 1000/1000 [00:43<00:00, 23.07it/s]


In [8]:
tagged_corpus_list[10]

TaggedDocument(words=['1', '재료', '를', '준비', '해주세요', '2', '시금치', '두', '단', '씻어서', '데칠', '준비', '해', '뒀네요', '시금치', '는', '다듬어서', '깨끗하게', '씻어서', '데', '쳐주세요', '3', '끓는', '물', '에', '소금', '넣고', '데', '쳐주면', '시금치', '가', '파릇파릇하게', '데', '쳐진답니다', '데친', '다음', '찬물', '에', '헹궈', '서', '소금', '참기름', '통깨', '마늘', '넣고', '무쳐주면', '되네요', '4', '평소', '에는', '계란', '지단', '얇게', '해서', '채', '썰어서', '넣어', '줬는데', '오늘', '은', '두', '툼', '하', '게', '부쳐', '봤어요', '5', '당근', '도', '채', '썰어서', '소금', '간', '해서', '볶아', '줬어요', '6', '햄', '도', '살짝', '볶아주고', '7', '우엉', '이', '있으면', '좋은데', '없어서', '대신', '어묵', '볶음', '넣어', '줬어요', '어묵', '볶다가', '간장', '물엿', '2', '개', '씩', '넣고', '볶아', '줬어요', '8', '김밥', '재료', '준비', '끝', '9', '밥', '에', '소금', '참기름', '통깨', '로', '간', '해줬고요', '10', '김위', '에', '밥', '얇게', '깔고', '재료', '순서대로', '쭉', '올려서', '돌돌', '말아주면', '되네요', '김발', '사용', '해도', '좋고', '저', '는', '그냥', '마는', '게', '편해서', '김발', '없이', '말', '아', '줬어요', '11', '참기름', '발라', '서', '잘라주기만', '하면', '되죠', '12', '언제', '먹어도', '질리', '지', '않는', '김밥', '완성', '김밥', '이랑', '잘', '어울리

In [9]:
from gensim.models import doc2vec

model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)

# 수정된 부분: doctags 대신 key_to_index 사용
print(f"Tag Size: {len(model.dv.key_to_index)}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=10)

# 모델 저장
model.save('recipe.doc2vec')


Tag Size: 998 / 

In [63]:
similar_doc = model.dv.most_similar('김밥 맛있게 싸는법~')
print(similar_doc)

[('흔한 물김치가 아닌 겉절이로 맛있게 돌나물오이김치', 0.7805804014205933), ('꼬마김밥', 0.7715111374855042), ('오징어볶음과 오징어덮밥으로도 간단하게~', 0.7562265396118164), ('가성비도 맛도 좋은 노브랜드 비빔냉면', 0.7396169304847717), ('꽁치 통조림으로 만든 꽁치 김치찜..', 0.7355816960334778), ('초 간단 소고기 볶음밥', 0.7337271571159363), ('한그릇요리 매콤 연어덮밥', 0.7315800189971924), ('스팸 치즈 컵밥..', 0.7225807309150696), ('아삭 건강한 깍두기 담그는법~', 0.7224217653274536), ('가지요리 : 간단하면서 고급스런 토마토가지볶음', 0.7109076380729675)]


In [64]:
similar_doc_2 = model.dv.most_similar('달콤한 애플 군만두')
print(similar_doc_2)

[('너무너무 간단하고 맛난 감자나물 비빔밥', 0.8792570233345032), ('한그릇요리 뚝배기 치즈밥 만들기', 0.8714075684547424), ('묵은지와 고등어 통조림으로 만든 묵은지 고등어조림', 0.8679327368736267), ('스파게티 떡볶이', 0.8428436517715454), ('곤약감자조림', 0.8392762541770935), ('영양많은 밑반찬 호두 간장 마늘조림', 0.8392329812049866), ('육수없이 끓이는 부대찌개', 0.8387001156806946), ('콩나물몰무침...콩나물&모자반 무침(겨울제철반찬)', 0.8284345865249634), ('볼어묵조림, 진하고 매운 볼어묵볶음 레시피 역대급이다', 0.8228673934936523), ('김치참치죽 끓이기', 0.8180534243583679)]


### 해결해야할 과제


1. 레시피에서 하는 요리에 대한 정의가 필요함
  - 레시피 타이틀 전처리
  - 레시피 태그 전처리 
2. 레시피에 따라서 2인분, 3인분, 4인분 다양함

In [65]:
data["recipe_summary1"].value_counts()

2인분       57219
4인분       31408
1인분       25937
3인분       19687
6인분 이상    16377
5인분        3670
Name: recipe_summary1, dtype: int64

### 영양소를 고려한 임베딩

In [177]:
toy = data[["recipe_title", "recipe_ingredients"]].iloc[:1000].copy()

In [178]:
# 식재료 칼럼 만들기 (20개)
for i in range(1, 21):
    toy.loc[:, f'ingredient{i}'] = None
    toy.loc[:, f'quantity{i}'] = None
    toy.loc[:, f'unit{i}'] = None


In [191]:
import tqdm
import re
import ast
non_matching_items =[]
for idx, row in tqdm.tqdm(toy.iterrows(), total=toy.shape[0]):
    ingredients_dict = ast.literal_eval(row["recipe_ingredients"])
    ingredient_count = 1
    for category, items in ingredients_dict.items():
        if items:  # 아이템이 존재하는 경우
            for item in items:
                # 숫자가 포함된 경우와 숫자가 없는 경우 모두 처리
                match = re.match(r'([가-힣]+(\([가-힣]+\))?)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item)
                if match:
                    ingredient, _, quantity, unit = match.groups()
                    
                    toy.at[idx, f'ingredient{ingredient_count}'] = ingredient
                    toy.at[idx, f'quantity{ingredient_count}'] = quantity if quantity else ""  # 숫자가 없는 경우 빈 문자열 할당
                    toy.at[idx, f'unit{ingredient_count}'] = unit if unit else ""  # 단위가 없는 경우 빈 문자열 할당

                    ingredient_count += 1
                else:
                    non_matching_items[idx] = item

for idx, item in non_matching_items.items():
    print(f'Row {idx}: {item}')



100%|██████████| 1000/1000 [00:03<00:00, 254.42it/s]

Row 34: ​고추가루1/2큰술
Row 258: (소주or맛술)1큰술
Row 302: ﻿대파조금
Row 367: 18배멸치육수2T
Row 424: ​간장2수저
Row 598: 
Row 837: 
Row 892: 
Row 895: ​식용유1큰술
